In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import csv

In [ ]:
transaction = pd.read_csv("transactions.csv", delimiter="|")
items = pd.read_csv("items.csv", delimiter="|")

In [ ]:
transaction.head()

In [ ]:
print(transaction.shape)
print(items.shape)

In [ ]:
data = pd.merge(transaction, items, on=["itemID"])

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
# choosing Target variable
#print(inner_merged_total['basket'].value_counts())
#print(inner_merged_total['click'].value_counts())
print(data['order'].value_counts())

In [ ]:
##data preprocessing 
##generate target variable interest
data["Interest"] = data["click"] + data["basket"]+ data["order"]

## dropping season ID input for dr raheem
del data["sessionID"]

## aggregrate (idea is to generate the frequencies of item id, and multiply it by interest, getting the total interest of the items)

In [ ]:
data.head()
#print(inner_merged_total['Interest'].value_counts())

## Group by item ID, and then count the item ID, and then interest * count of itemID = total interest column

In [ ]:
##generate frequency
test = data.value_counts(['itemID']).reset_index(name='Frequency')
print(test.head())

In [ ]:
#Merging frequency 
book_data = pd.merge(data, test, on=["itemID"])
book_data.head()

In [ ]:
#remove dupicate rows 
print(book_data.shape)
book_data.drop_duplicates(subset ="itemID", keep='first' ,inplace=True)
print(book_data.shape)

In [ ]:
book_data.head()

In [ ]:
##compute total interest (interest*frequency)

In [ ]:
##Generate total interest

book_data["Total_Interest"] = book_data["Frequency"] * book_data["Interest"] 

##Print latest book_data

book_data 


In [ ]:
##Count total number for frequency and Total Interest
print(book_data['Total_Interest'].value_counts())


In [ ]:
#Latest Data types
book_data.dtypes



In [ ]:
##EDA for Click basket and order (any order without click find out), Make a pair plot using seaborn

In [ ]:
##Create new column where order != 0 but basket and/or click is 0

def identify(book_data):

    if (book_data['order'] != 0 and (book_data['basket']  | book_data['click'] == 0)):
        return 'Found'
    else:
        return 'Not found'

book_data['test'] = book_data.apply(identify, axis = 1)

book_data.head(10)

In [ ]:
#Print how many counts for order column

print(book_data['order'].value_counts())


In [ ]:
## Print how many counts for test column

print(book_data['test'].value_counts())


In [ ]:
## Print column where test == found

test_column = book_data.loc[book_data['test'] == 'Found']

test_column.head(10)


In [ ]:
##clone dataframe
sea_born_df = book_data.copy(deep=True)

In [ ]:
sea_born_df.shape

In [ ]:
## dropping unnessesary columns for pairplot
sea_born_df = sea_born_df.filter(items=['click', 'basket', 'order'])


In [ ]:
## Make pair plot using seaborn
import seaborn as sns


# sns.set(style="ticks", color_codes=True)
sns.pairplot(sea_born_df)


import matplotlib.pyplot as plt
plt.show()



In [ ]:
#Create histogram -> edited

x = book_data['Total_Interest'].value_counts()

plt.hist(x, bins = 60)
plt.show()